<a href="https://colab.research.google.com/github/nikibhatt/DS-Unit-1-Sprint-3-Linear-Algebra/blob/master/NB_Portfolio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200> 

In [4]:
df = pd.read_csv('googleplaystore.csv')
print(df.shape)
df.head()

(10841, 13)


,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Hypothesis: Number of installs for most popular(top 10) 'Teen' apps will far exceed the number of installs for most popular(top 10) 'non-Teen' apps

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error # 0.3 error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [0]:
df_original = df

# Data cleaning for "Size" column
df['Size'] = df['Size'].map(lambda x: x.rstrip('M'))
df['Size'] = df['Size'].map(lambda x: str(round((float(x.rstrip('k'))/1024), 1)) if x[-1]=='k' else x)
df['Size'] = df['Size'].map(lambda x: np.nan if x.startswith('Varies') else x)
# Data cleaning for "Installs" column
df['Installs'] = df['Installs'].map(lambda x: x.rstrip('+'))
df['Installs'] = df['Installs'].map(lambda x: ''.join(x.split(',')))
# Data cleaning for "Price" column
df['Price'] = df['Price'].map(lambda x: x.lstrip('$').rstrip())
# Row 10472 removed due to missing value of Category
df.drop(df.index[10472], inplace=True)
# Row [7312,8266] removed due to "Unrated" value in Content Rating
df.drop(df.index[[7312,8266]], inplace=True)

In [9]:
df.head(10)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19,10000,Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
35,How to draw Ladybug and Cat Noir,ART_AND_DESIGN,3.8,564,9.2,100000,Free,0,Everyone,Art & Design,"July 11, 2018",2.1,4.1 and up
36,UNICORN - Color By Number & Pixel Art Coloring,ART_AND_DESIGN,4.7,8145,24,500000,Free,0,Everyone,Art & Design;Creativity,"August 2, 2018",1.0.9,4.4 and up
37,Floor Plan Creator,ART_AND_DESIGN,4.1,36639,NaN,5000000,Free,0,Everyone,Art & Design,"July 14, 2018",Varies with device,2.3.3 and up
38,PIP Camera - PIP Collage Maker,ART_AND_DESIGN,4.7,158,11,10000,Free,0,Everyone,Art & Design,"November 29, 2017",1.3,4.0.3 and up
39,How To Color Disney Princess - Coloring Pages,ART_AND_DESIGN,4.0,591,9.4,500000,Free,0,Everyone,Art & Design,"March 31, 2018",1,4.0 and up
40,Drawing Clothes Fashion Ideas,ART_AND_DESIGN,4.2,117,15,10000,Free,0,Everyone,Art & Design,"July 20, 2018",2.0.1,4.0.3 and up
41,Sad Poetry Photo Frames 2018,ART_AND_DESIGN,4.5,176,10,100000,Free,0,Everyone,Art & Design,"April 2, 2018",1.0,4.0.3 and up
42,Textgram - write on photos,ART_AND_DESIGN,4.4,295221,NaN,10000000,Free,0,Everyone,Art & Design,"July 30, 2018",Varies with device,Varies with device
43,Paint Splash!,ART_AND_DESIGN,3.8,2206,1.2,100000,Free,0,Everyone,Art & Design;Creativity,"April 15, 2018",1.46,4.1 and up


In [0]:
# Sort by "Category"
df.sort_values("Category", inplace = True)